### Taylor Diagram Statistics

In [2]:
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import random
import numpy as np
import xarray as xr
import pandas as pd
import joblib
import pickle
import xesmf as xe
import glob

import seaborn as sns
import cmocean as cm            # really nice colorbars
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats

In [7]:
# Set these:
run_dir = '/data/artemis/workspace/vbennington/SOCAT_ML/pCO2_DIC'
approach = 'xg'
recon = 'pCO2_recon'
truth = 'pCO2'
start_yr = 1985
end_yr = 2018

# These change based on what you set above"
recon_dir = f"{run_dir}/models/reconstructions/{approach}"
recon_fname = f"{recon_dir}/{approach}_recon_pCO2_2D_mon_run1_1x1_198201-201912.nc"
recon_fname = f"{recon_dir}/{approach}_recon_pCO2_2D_mon_mean_1x1_198201-201912.nc"

In [8]:
# Grab reconstruction:
recon_product = xr.open_dataset(f"{recon_fname}")[recon].transpose('time','ylat','xlon')

In [9]:
# Set up dataframe #
####################
df = pd.DataFrame() # create empty data frame
df['stats']= ["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT","R_SOCCOM","STD_SOCCOM","RMSE_SOCCOM", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]
df.set_index([pd.Index(["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT","R_SOCCOM","STD_SOCCOM","RMSE_SOCCOM", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]), 'stats'])

# other approaches and their target variables # 
products = ['csirml6','jenamls','lsceffnn2','mpisomffn','HPD']

In [16]:
# My Reconstruction Stats #
###########################
R_BATS, STD_BATS, RMSE_BATS, b_obs = BATS_stats(recon_product,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, h_obs = HOT_stats(recon_product,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, s_obs = SOCCOM_stats(recon_product,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, l_obs = LDEO_stats(recon_product,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, g_obs = GLODAP_stats(recon_product,recon,start_yr=start_yr,end_yr=end_yr)

# Write to DataFrame:
df['pCO2-recon']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df['observed']=[1,b_obs,0,1,h_obs,0,1,s_obs,0,1,l_obs,0,1,g_obs,0]

/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360_peri.nc 
 You can set reuse_weights=True to save computing time.
Overwrite existing file: bilinear_180x360_180x360_peri.nc 
 You can set reuse_weights=True to save computing time.


In [17]:
df.head(20)

,stats,pCO2-recon,observed
0,R_BATS,0.932961,1.000000
1,STD_BATS,31.044342,35.604917
2,RMSE_BATS,13.027858,0.000000
3,R_HOT,0.912902,1.000000
4,STD_HOT,19.271492,19.401837
5,RMSE_HOT,9.067840,0.000000
6,R_SOCCOM,0.639642,1.000000
7,STD_SOCCOM,18.667327,34.874618
8,RMSE_SOCCOM,27.952882,0.000000
9,R_LDEO,0.902838,1.000000


In [18]:
# Get Luke's runs
#-----------------------------------------------------------------------------------------
# load pco2 data
#-----------------------------------------------------------------------------------------
data_dir = '/local/data/artemis/workspace/gloege/data/LDEO-HPD'
ds_spco2 = xr.merge([xr.open_dataset(fl) for fl in glob.glob(f'{data_dir}/XGB*.nc')])
# ds_spco2 = ds_spco2.sel(time=slice("1985","2018"))
# GCB 2020
variables = ['corrected_cesm_spco2_1x1_A','corrected_csiro_spco2_1x1_A','corrected_fesom_spco2_1x1_A','corrected_mpi_spco2_1x1_A','corrected_cnrm_spco2_1x1_A',
    'corrected_ipsl_spco2_1x1_A',
    'corrected_planktom_spco2_1x1_A',
    'corrected_noresm_spco2_1x1_A',
    'corrected_princeton_spco2_1x1_A',]
# make dummy variable of zeros
ds_spco2['spco2'] = ds_spco2['corrected_cesm_spco2_1x1_A']*0
# add all predictions together
for var in variables:
    ds_spco2['spco2'] += ds_spco2[f'{var}']
# divide by number vars to average
ds_spco2['spco2'] = ds_spco2['spco2'] / len(variables)

#replace longitude from 0-360 to -180 to 180
ds_spco2['lon'] = list(map(lambda x: x-360 if x>180 else x, ds_spco2['lon'].values))

# Sort by longitude
ds_spco2 = ds_spco2.sortby('lon')
ds_spco2['time']=pd.date_range(start=f'1982-01T00:00:00.000000000',end=f'2018-12T00:00:00.000000000',freq='MS') + np.timedelta64(14, 'D')

In [71]:
# Grab the other standardised pCO2 products #
products = ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','HPD']
socom = xr.open_dataset('/data/artemis/observations/SOCOM/extra_files/SeaFlux_v2021.04_spco2_SOCOM_unfilled_1982-2019.nc')

In [72]:
# Calculate statistics for the standardized products:

for i in range(0,len(products)): 
    
    apr = products[i]
    
    if apr == 'HPD':
        gridded = ds_spco2.spco2[36::,:,:]
        recon = 'spco2'
    else:
        gridded = socom[apr]
        recon = apr
    
    R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

    R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

    R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

    R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

    R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)
    # Write to DataFrame:
    df[apr]=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]

/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360_peri.nc 
 You can set reuse_weights=True to save computing time.


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in greater


LDEO OBS # = 177589
Overwrite existing file: bilinear_180x360_180x360_peri.nc 
 You can set reuse_weights=True to save computing time.
0.6804447101586713


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in greater


In [21]:
df.head(20)

,stats,pCO2-recon,observed,CSIR_ML6,JENA_MLS,JMA_MLR,MPI_SOMFFN,NIES_FNN,CMEMS_FFNN,HPD
0,R_BATS,0.932961,1.000000,0.943677,0.904462,0.930730,0.930467,0.933019,0.940411,0.947651
1,STD_BATS,31.044342,35.604917,30.124058,33.698700,25.783770,29.681829,30.602938,28.822685,33.184704
2,RMSE_BATS,13.027858,0.000000,12.303783,16.071000,15.126386,13.561577,13.631888,13.106187,11.504445
3,R_HOT,0.912902,1.000000,0.899761,0.868891,0.874907,0.841460,0.842609,0.890141,0.924602
4,STD_HOT,19.271492,19.401837,17.239357,21.909908,24.088194,17.039692,17.217947,19.167067,19.381221
5,RMSE_HOT,9.067840,0.000000,8.699241,11.330748,11.942700,10.551415,10.764331,9.682446,9.307822
6,R_SOCCOM,0.639642,1.000000,0.633211,0.461050,0.587354,0.594642,0.511610,0.549838,0.612133
7,STD_SOCCOM,18.667327,34.874618,17.089645,22.455618,18.700811,18.507807,16.979551,18.315603,18.667810
8,RMSE_SOCCOM,27.952882,0.000000,28.636909,32.617776,28.388938,28.683655,32.681202,30.011610,28.366579
9,R_LDEO,0.902838,1.000000,0.859845,0.860019,0.806989,0.865779,0.725381,0.844770,0.889034


In [66]:
# CarboScope 2021:
start_yr = 1985
end_yr = 2018
gridded = xr.open_dataset("/data/artemis/simulations/CarboScope/CarboScope_pCO2_monthly_1x1_1957-2020.nc").pCO2
gridded['time'] =  pd.date_range(start='1957-01-01T00:00:00.000000000', 
                      end='2020-12-01T00:00:00.000000000',freq='MS') + np.timedelta64(14, 'D')
gridded = gridded.rename({'lat':'ylat','lon':'xlon'})
#gridded_masked = gridded.where(~np.isnan(recon_product))

recon = 'pCO2'

R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

# Are we getting low correlations with LDEO because CarboScope includes poles? We get an even lower correlation for LDEO when we mask out the areas
# not reconstructed by our reconstruction... so i guess not
#R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded_masked,recon,start_yr=start_yr,end_yr=end_yr), R = 0.56
R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

# Write to DataFrame:
df['CarboScope']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]

/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360_peri.nc 
 You can set reuse_weights=True to save computing time.


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in greater


LDEO OBS # = 175111
Overwrite existing file: bilinear_180x360_180x360_peri.nc 
 You can set reuse_weights=True to save computing time.


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in greater


In [67]:
df.head(20)

,stats,pCO2-recon,observed,CSIR_ML6,JENA_MLS,JMA_MLR,MPI_SOMFFN,NIES_FNN,CMEMS_FFNN,HPD,CarboScope
0,R_BATS,0.932961,1.000000,0.943677,0.904462,0.930730,0.930467,0.933019,0.940411,0.947651,0.904606
1,STD_BATS,31.044342,35.604917,30.124058,33.698700,25.783770,29.681829,30.602938,28.822685,33.184704,35.975444
2,RMSE_BATS,13.027858,0.000000,12.303783,16.071000,15.126386,13.561577,13.631888,13.106187,11.504445,15.833943
3,R_HOT,0.912902,1.000000,0.899761,0.868891,0.874907,0.841460,0.842609,0.890141,0.924602,0.867030
4,STD_HOT,19.271492,19.401837,17.239357,21.909908,24.088194,17.039692,17.217947,19.167067,19.381221,22.306998
5,RMSE_HOT,9.067840,0.000000,8.699241,11.330748,11.942700,10.551415,10.764331,9.682446,9.307822,11.660943
6,R_SOCCOM,0.639642,1.000000,0.633211,0.461050,0.587354,0.594642,0.511610,0.549838,0.612133,0.420316
7,STD_SOCCOM,18.667327,34.874618,17.089645,22.455618,18.700811,18.507807,16.979551,18.315603,18.667810,23.800142
8,RMSE_SOCCOM,27.952882,0.000000,28.636909,32.617776,28.388938,28.683655,32.681202,30.011610,28.366579,33.769735
9,R_LDEO,0.902838,1.000000,0.859845,0.860019,0.806989,0.865779,0.725381,0.844770,0.889034,0.859807


In [68]:
# Write out Files #
df.to_csv(f'{recon_dir}/ens_pCO2_Taylor_stats_{start_yr}-{end_yr}.csv',index=False)

In [15]:
def BATS_stats(ml_timeseries,recon,start_yr=1984,end_yr=2018,blat=121,blon=115):
    
    # determine appropriate START YEAR # 
    if start_yr < 1984:
        start_yr = 1984
  
    if end_yr > 2018:
        end_yr = 2018
    
    # Grab BATS observations:
    bats1 = xr.open_dataset('/data/artemis/workspace/gloege/data/BATS/BATS.nc')
    bats2 = xr.open_dataset('/data/artemis/observations/BATS/2021_data/bats_spco2_199110-201906.nc')
    bats = xr.concat(objs=[bats1,bats2],dim='time')
    # Remove duplicates
    bats = bats.groupby('time').mean()
    # Put BATS monthly averages:
    bats_monthly = bats.spco2.to_dataframe().resample('M').mean()
    # Extract years of interest:
    bats_monthly = bats_monthly.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    # 31 50'N 64 10'W ###########     
    
    # Model already monthly #
    pco2_df = ml_timeseries[:,blat,blon].squeeze().to_dataframe()
    pco2_df = pco2_df.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-15 00:00:00']    
    
    pco2_df.head()
    # get time uniform:
    bats_monthly = bats_monthly.set_index(pco2_df.index)
    
    # correlate where no NANs
    fib = (~np.isnan(bats_monthly.spco2) & ~np.isnan(pco2_df[recon]))
    R = np.corrcoef(pco2_df[recon][fib],bats_monthly.spco2[fib])[0,1]
    STD = np.nanstd(pco2_df[recon][fib])
    BATS_STD = np.nanstd(bats_monthly.spco2[fib])
    
    #RMSE
    RMSE = np.sqrt(np.square(bats_monthly.spco2[fib]-pco2_df[recon][fib]).sum()/(fib.sum()))
    
    return R, STD, RMSE, BATS_STD

In [14]:
def HOT_stats(ml_timeseries,recon,start_yr=1989,end_yr=2016,hlat=112,hlon=22):
    
    if start_yr < 1989:
        start_yr = 1989
    if end_yr < start_yr:
        print("Your dates are wrong for HOT; data starts in 1989")
    if end_yr > 2016:
        end_yr = 2016
    
    # Grab observations
    hot = xr.open_dataset('/data/artemis/workspace/gloege/data/HOT/HOT.nc')
    hot=hot.where(hot.spco2>0)
    
    pco2_df = ml_timeseries[:,hlat,hlon].squeeze().to_dataframe()
    pco2_df = pco2_df.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-15 00:00:00']
    
    
    # P
    hots_monthly = hot.spco2.to_dataframe().resample('M').mean()
    hots_monthly = hots_monthly.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    hots_monthly = hots_monthly.set_index(pco2_df.index)
    # Point by point comparison #
    ind = ((~np.isnan(hots_monthly.spco2)) & (hots_monthly.spco2 > 150) & (~np.isnan(pco2_df[recon])))
    R = np.corrcoef(pco2_df[recon][ind],hots_monthly.spco2[ind])[0,1]
    
    STD = np.std(pco2_df[recon][ind])
    HOT_STD = np.nanstd(hots_monthly.spco2[ind])
    
    #RMSE
    RMSE = np.sqrt(np.square(hots_monthly.spco2[ind]-pco2_df[recon][ind]).sum()/ind.sum())
    
    # We want to know the correlation coef, STD of reconstruction, trendline, seasonal cycle:
    return R, STD, RMSE, HOT_STD

In [13]:
def SOCCOM_stats(ml_gridded,recon,start_yr=1982,end_yr=2019):
    
    start_yr = max(1985,start_yr)
    if recon in ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','spco2']:
        end_yr = 2018
        
    # grab observations and years of interest #
    df = xr.open_dataset('/data/artemis/observations/SOCCOM/processed/SOCCOM_gridded_spco2_mon_195001-202112.nc') 
    spco2 = df.spco2[(start_yr-1950)*12:(end_yr-1949)*12,:,:]
    time = df.time[(start_yr-1950)*12:(end_yr-1949)*12]
    lat = df.ylat
    lon = df.xlon
   
    pco2 = ml_gridded.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr+1}-1-01']
    
    # Grab from model gridded reconstruction #
    if recon in ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','spco2']:
        lon = ml_gridded.lon
        lat = ml_gridded.lat
        pco2_stack = pco2.stack(level=['time','lat','lon'])
    else:
        ylat = ml_gridded.ylat
        xlon = ml_gridded.xlon
        pco2_stack = pco2.stack(level=['time','ylat','xlon'])
    
    # stack the time series #
    soccom = spco2.stack(level=['time','ylat','xlon'])
    
    fis = ((~np.isnan(soccom.values)) & (~np.isnan(pco2_stack.values)) & (soccom.values < 815) & (soccom.values > 150))
    
    R = np.corrcoef(soccom[fis],pco2_stack[fis])[0,1]
    STD = np.std(pco2_stack[fis].values)
    SOCCOM_STD = np.std(soccom[fis].values)
    
    #RMSE
    RMSE = np.sqrt(np.square(soccom[fis].values-pco2_stack[fis].values).sum()/fis.sum())
    
    return R, STD, RMSE, SOCCOM_STD

In [63]:
def LDEO_stats(ml_gridded,recon,start_yr=1982,end_yr=2018):
    
    start_yr = max(start_yr,1985)
    if end_yr > 2018:
        end_yr = 2018

    if recon in ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','spco2']:
        lon = ml_gridded.lon
        lat = ml_gridded.lat
    else:
        lon = ml_gridded.xlon
        lat = ml_gridded.ylat
        
    mtime = ml_gridded.time

    # Load the observations #
    ldeo = xr.open_dataset('/data/artemis/observations/LDEO_database/processed/LDEOv2018_1x1_198201-201812.nc')
    ldeo_pco2=ldeo.spco2_mean[(start_yr-1982)*12:(end_yr-1981)*12,:,:]
    ldeo_lat=ldeo.lat
    ldeo_lon=ldeo.lon
    ldeo_time = ldeo.time
    
    # Regrid to LDEO's grid #########
    mgrid = xr.Dataset({'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    lgrid = xr.Dataset({'lat':(['lat'],ldeo_lat.values),'lon':(['lon'],ldeo_lon.values)})
    mpco2 = xr.Dataset({recon:(['time','lat','lon'],ml_gridded.values),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    regridder = xe.Regridder(mpco2, lgrid, 'bilinear','periodic')
    pco2_new = regridder(ml_gridded)

    ldeo_pco2 = ldeo_pco2.where(((ldeo_pco2<850) & (ldeo_pco2>150)))
    ldeo_pco2 = ldeo_pco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    pco2_new = pco2_new.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    

    # stack
    lpco2 = ldeo_pco2.stack(level=['time','lat','lon'])
    pco2_stack = pco2_new.stack(level=['time','lat','lon'])
    
    indx = ((~np.isnan(lpco2.values)) & (~np.isnan(pco2_stack.values)) & (pco2_stack.values > 50))
    print("LDEO OBS # =",sum(indx))

    R = np.corrcoef(lpco2[indx],pco2_stack[indx])[0,1]
    LDEO_STD = np.std(lpco2[indx].values)
    STD = np.std(pco2_stack[indx].values)
    #RMSE
    RMSE = np.sqrt(np.square(lpco2[indx].values-pco2_stack[indx].values).sum()/indx.sum())
    
    return R, STD, RMSE, LDEO_STD

In [65]:
def GLODAP_stats(ml_gridded,recon,start_yr=1982,end_yr=2018):
    
    # Reconstructions #
    if recon in ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','spco2']:
        lon = ml_gridded.lon
        lat = ml_gridded.lat
        start_yr = max(start_yr,1985)
    else:
        lon = ml_gridded.xlon
        lat = ml_gridded.ylat
        
    start_yr = max(start_yr,1985)
    end_yr = min(end_yr,2018)
    
    mtime = ml_gridded.time
    
    # Load the data #
    glod = xr.open_dataset('/data/artemis/observations/GLODAP_v2/processed/GLODAPv2_spco2_1x1_198201-201812.nc')
    glod_pco2=glod.spco2_mean
    glod_lat=glod.lat
    glod_lon=glod.lon
    glod_time = glod.time
    
    # Deal with NaNs #
    tmp = ml_gridded.where(ml_gridded > 0)
    tmp = tmp.where(tmp < 850)
    
    # Regrid to GLODAP grid #
    mgrid = xr.Dataset({'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    ggrid = xr.Dataset({'lat':(['lat'],glod_lat.values),'lon':(['lon'],glod_lon.values)})
    mpco2 = xr.Dataset({recon:(['time','lat','lon'],tmp),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    regridder = xe.Regridder(mpco2, ggrid, 'bilinear','periodic')
    pco2_new = regridder(tmp)
    
    # Extract Time #
    glod_pco2 = glod_pco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    glod_pco2 = glod_pco2.where(((glod_pco2 >200) & (glod_pco2 < 600)))
    pco2_new = pco2_new.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    
    # Stack
    gpco2_stack = glod_pco2.stack(level=['time','lat','lon'])
    pco2_stack = pco2_new.stack(level=['time','lat','lon'])
    
    # Stats
    indx = ((~np.isnan(gpco2_stack.values)) & (~np.isnan(pco2_stack.values)) & (pco2_stack.values>50))
    R = np.corrcoef(gpco2_stack[indx],pco2_stack[indx])[0,1]
    GLODAP_STD = np.std(gpco2_stack[indx].values)
    STD = np.std(pco2_stack[indx].values)
    #RMSE
    RMSE = np.sqrt(np.square(gpco2_stack[indx].values-pco2_stack[indx].values).sum()/indx.sum())
    
    return R, STD, RMSE, GLODAP_STD